# Leetcode 000-099
See https://leetcode.com/problems/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import heapq
from typing import *

In [ ]:
# utility functions and version check

def run_sample(obj, sample, funcname):
    lines=[s for s in sample.splitlines() if len(s)>0 ]
    data=[json.loads(s) for s in lines]
    func=getattr(obj, funcname)
    res=func(*data[:-1])
    if res!=data[-1]:
        raise ValueError(f'expected {data[-1]}, actual {res}')

print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # leetcode nnn)

In [ ]:
# leetcode 001
# https://leetcode.com/problems/two-sum/

sample1='''
[2,7,11,15]
9
[0,1]
'''

sample2='''
[3,2,4]
6
[1,2]
'''

sample3='''
[3,3]
6
[0,1]
'''

class Solution:
    # leetcode 001, idea: optimized solution involves a map of number to set of position indices
    # as well as the numbers as set, and distinction between two cases; two numbers are the same or not
    def twoSum(self, nums: List[int], target: int) -> List[int]:
        poz={}
        for i,v in enumerate(nums):
            ps=poz.setdefault(v, set())
            ps.add(i)
        numz=set(nums)
        #print(poz, numz)
        for n in numz:
            if target-n in numz:
                if n==target-n:
                    ps=poz.get(n)
                    if len(ps)>1:
                        return list(ps)[:2]
                else:
                    ps1=poz.get(n)
                    ps2=poz.get(target-n)
                    return [list(ps1)[0], list(ps2)[0]]
        assert False

run_sample(Solution(), sample1, 'twoSum')
run_sample(Solution(), sample2, 'twoSum')
run_sample(Solution(), sample3, 'twoSum')

In [ ]:
# leetcode 002
# https://leetcode.com/problems/add-two-numbers/

sample1='''
[2,4,3]
[5,6,4]
[7,0,8]
'''

sample2='''
[0]
[0]
[0]
'''

sample3='''
[9,9,9,9,9,9,9]
[9,9,9,9]
[8,9,9,9,0,0,0,1]
'''

# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

class Solution:
    def addTwoNumbers2(self, l1: List[int], l2: List[int]) -> List[int]:
        y=None
        for n in l1[::-1]:
            x=ListNode(n, y)
            y=x
        l1x=y
        y=None
        for n in l2[::-1]:
            x=ListNode(n, y)
            y=x
        l2x=y
        rx=self.addTwoNumbers(l1x, l2x)
        res=[]
        while rx is not None:
            res.append(rx.val)
            rx=rx.next
        return res

    # leetcode 2
    def addTwoNumbers(self, l1: Optional[ListNode], l2: Optional[ListNode]) -> Optional[ListNode]:
        rem=0
        y0=None
        y=None
        while not (l1 is None and l2 is None and rem==0):
            n=rem+(0 if l1 is None else l1.val)+(0 if l2 is None else l2.val)
            rem=0
            if n>9:
                rem=1
                n-=10
            x=ListNode(n, None)
            if y is None:
                y0=x
            else:
                y.next=x         
            y=x
            l1=l1.next if l1 is not None else None
            l2=l2.next if l2 is not None else None
        return y0

run_sample(Solution(), sample1, 'addTwoNumbers2')
run_sample(Solution(), sample2, 'addTwoNumbers2')
run_sample(Solution(), sample3, 'addTwoNumbers2')

In [ ]:
# leetcode 003
# https://leetcode.com/problems/longest-substring-without-repeating-characters/

sample1='''
"abcabcbb"
3
'''

sample2='''
"bbbbb"
1
'''

sample3='''
"pwwkew"
3
'''

sample4='''
""
0
'''

class Solution:
    # leetcode 003, idea: solution with a set as a moving window
    def lengthOfLongestSubstring(self, s: str) -> int:
        maxlen=0
        if len(s)<1:
            return maxlen
        cs=set() # set of chars
        i=0 # start of win that is considered
        j=0 # one beyond end of win that is considered
        while True:
            while j<len(s) and s[j] not in cs:
                cs.add(s[j])
                j+=1
            if len(cs)>maxlen:
                maxlen=len(cs)
            assert s[i] in cs
            cs.remove(s[i])
            i+=1
            assert j>=i
            if i>=len(s):
                break
        return maxlen

run_sample(Solution(), sample1, 'lengthOfLongestSubstring')
run_sample(Solution(), sample2, 'lengthOfLongestSubstring')
run_sample(Solution(), sample3, 'lengthOfLongestSubstring')
run_sample(Solution(), sample4, 'lengthOfLongestSubstring')

In [ ]:
# leetcode 004
# https://leetcode.com/problems/median-of-two-sorted-arrays/

sample1='''
[1,3]
[2]
2.00000
'''

sample2='''
[1,2]
[3,4]
2.50000
'''

class Solution:
    # leetcode 004
    def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
        combined=list(nums1)
        combined.extend(nums2)
        combined=sorted(combined)
        ln=len(combined)
        if ln%2==1:
            return combined[ln//2]
        else:
            return (combined[ln//2]+combined[ln//2-1])/2

run_sample(Solution(), sample1, 'findMedianSortedArrays')
run_sample(Solution(), sample2, 'findMedianSortedArrays')

In [ ]:
# leetcode 005
# https://leetcode.com/problems/longest-palindromic-substring/

sample1='''
"babad"
"bab"
'''

sample2='''
"cbbd"
"bb"
'''

sample3='''
"abcd"
"a"
'''

class Solution:
    # leetcode 005, idea: test each position in s as middle of the palindrome, of even or uneven length,
    # keep longest
    def longestPalindrome(self, s: str) -> str:
        longest=''
        for i in range(len(s)):
            # test uneven
            j=i # first char pos. of palindrome
            k=i # last char pos. of palindrome, inclusive
            while j-1>=0 and k+1<len(s) and s[j-1]==s[k+1]:
                j-=1
                k+=1
            if k+1-j>len(longest):
                longest=s[j:k+1]
            # test even
            j=i
            k=i-1
            while j-1>=0 and k+1<len(s) and s[j-1]==s[k+1]:
                j-=1
                k+=1
            if k+1-j>len(longest):
                longest=s[j:k+1]
        return longest

run_sample(Solution(), sample1, 'longestPalindrome')
run_sample(Solution(), sample2, 'longestPalindrome')
run_sample(Solution(), sample3, 'longestPalindrome')

In [ ]:
# leetcode 007
# https://leetcode.com/problems/reverse-integer/

sample1='''
123
321
'''

sample2='''
-123
-321
'''

sample3='''
120
21
'''

sample4='''
1534236469
0
'''

class Solution:
    # leetcode 007
    def reverse(self, x: int) -> int:
        s=str(x)[::-1]
        if s[-1]=='-':
            s='-'+s[:-1]
        res=int(s)
        maxnum=1<<31
        if not (-maxnum <= res <= maxnum-1):
            return 0
        return res

run_sample(Solution(), sample1, 'reverse')
run_sample(Solution(), sample2, 'reverse')
run_sample(Solution(), sample3, 'reverse')
run_sample(Solution(), sample4, 'reverse')

In [ ]:
# leetcode 008
# https://leetcode.com/problems/string-to-integer-atoi/

sample1='''
"42"
42
'''

sample2='''
"   -42"
-42
'''

sample3='''
"4193 with words"
4193
'''

sample4='''
" a"
0
'''

sample5='''
""
0
'''

sample6='''
"+-12"
0
'''

class Solution:
    # leetcode 008
    def myAtoi(self, s: str) -> int:
        i=0 # search for start of number
        while i<len(s) and s[i]==' ':
            i+=1
        if i>=len(s):
            return 0
        j=i # search for end of number
        if s[j]=='-' or s[j]=='+':
            j+=1
        while j<len(s) and ('0' <= s[j] <= '9'):
            j+=1
        ns=s[i:j]
        if len(ns)<1 or ns=='-' or ns=='+':
            return 0
        n=int(ns)
        maxval=1<<31
        if n<-maxval:
            n= -maxval
        if n>maxval-1:
            n=maxval-1
        return n

run_sample(Solution(), sample1, 'myAtoi')
run_sample(Solution(), sample2, 'myAtoi')
run_sample(Solution(), sample3, 'myAtoi')
run_sample(Solution(), sample4, 'myAtoi')
run_sample(Solution(), sample5, 'myAtoi')
run_sample(Solution(), sample6, 'myAtoi')

In [ ]:
# leetcode 013
# https://leetcode.com/problems/roman-to-integer/

sample1='''
"III"
3
'''

sample2='''
"LVIII"
58
'''

sample3='''
"MCMXCIV"
1994
'''

class Solution:
    # leetcode 013
    syms={
        'I': 1,
        'V': 5,
        'X': 10,
        'L': 50,
        'C': 100,
        'D': 500,
        'M': 1000,
    }
    def romanToInt(self, s: str) -> int:
        tot=0
        for i in range(len(s)):
            v1=self.syms[s[i]]
            v2=self.syms[s[i+1]] if i+1<len(s) else 0
            if v1<v2:
                tot-=v1
            else:
                tot+=v1
        return tot

run_sample(Solution(), sample1, 'romanToInt')
run_sample(Solution(), sample2, 'romanToInt')
run_sample(Solution(), sample3, 'romanToInt')

In [ ]:
# leetcode 014
# https://leetcode.com/problems/longest-common-prefix/

sample1='''
["flower","flow","flight"]
"fl"
'''

sample2='''
["dog","racecar","car"]
""
'''

sample3='''
[""]
""
'''

class Solution:
    # leetcode 014
    def longestCommonPrefix(self, strs: List[str]) -> str:
        res=""
        i=0
        while True:
            for s in strs:
                if len(s)<=i:
                    return res
                if s[i]!=strs[0][i]:
                    return res
            res+=strs[0][i]
            i+=1

run_sample(Solution(), sample1, 'longestCommonPrefix')
run_sample(Solution(), sample2, 'longestCommonPrefix')
run_sample(Solution(), sample3, 'longestCommonPrefix')

In [ ]:
# leetcode 015
# https://leetcode.com/problems/3sum/

sample1='''
[-1,0,1,2,-1,-4]
[[-1,-1,2],[-1,0,1]]
'''

sample2='''
[0,1,1]
[]
'''

sample3='''
[0,0,0]
[[0,0,0]]
'''

sample4='''
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
[[0,0,0]]
'''

class Solution:
    # leetcode 015, idea: put negative value of each item in a map (to list of indices), then loop i,j through
    # the array, look for matches with different indices
    def threeSum(self, nums: List[int]) -> List[List[int]]:
        vals={} # maps nums element to list of indices
        res=set() # set of sorted triplet tuples - of the values
        for i,v in enumerate(nums):
            vl=vals.setdefault(-v, [])
            vl.append(i)
        for i in range(len(nums)):
            num_i=nums[i]
            for j in range(i+1, len(nums)):
                num_j=nums[j]
                if (num_i+num_j) in vals:
                    vl=vals[num_i+num_j]
                    for k in vl:
                        if k!=i and k!=j:
                            num_k=nums[k]
                            # to check unique triplets of values we sort the triplet and
                            # stick it in the set named res,
                            # for speed we're unrolling the sort here :/
                            if num_i<num_j:
                                if num_k<num_i:
                                    res.add( (num_k, num_i, num_j) )
                                elif num_k>num_j:
                                    res.add( (num_i, num_j, num_k) )
                                else:
                                    res.add( (num_i, num_k, num_j) )
                            else: # num_i >= num_j
                                if num_k<num_j:
                                    res.add( (num_k, num_j, num_i) )
                                elif num_k>num_i:
                                    res.add( (num_j, num_i, num_k) )
                                else:
                                    res.add( (num_j, num_k, num_i) )                                
                            break
        res=sorted([ list(x) for x in list(res) ])
        return res

run_sample(Solution(), sample1, 'threeSum')
run_sample(Solution(), sample2, 'threeSum')
run_sample(Solution(), sample3, 'threeSum')
run_sample(Solution(), sample4, 'threeSum')

In [ ]:
# leetcode 019
# https://leetcode.com/problems/remove-nth-node-from-end-of-list/

sample1='''
[1,2,3,4,5]
2
[1,2,3,5]
'''

sample2='''
[1]
1
[]
'''

sample3='''
[1,2]
1
[1]
'''

sample4='''
[1,2]
2
[2]
'''

# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

class Solution:
    def removeNthFromEnd2(self, l1: List[int], nnn: int) -> List[int]:
        y=None
        for n in l1[::-1]:
            x=ListNode(n, y)
            y=x
        l1x=y
        rx=self.removeNthFromEnd(l1x, nnn)
        res=[]
        while rx is not None:
            res.append(rx.val)
            rx=rx.next
        return res

    # leetcode 019
    def removeNthFromEnd(self, head: Optional[ListNode], n: int) -> Optional[ListNode]:
        idx={}
        node=head
        i=0
        while node is not None:
            idx[i]=node
            i+=1
            node=node.next
        i=len(idx)-n # node to remove
        if i<1:
            if len(idx)<2:
                return None
            else:
                return idx[1]
        else:
            node=idx[i-1]
            node.next=node.next.next
            return head

run_sample(Solution(), sample1, 'removeNthFromEnd2')
run_sample(Solution(), sample2, 'removeNthFromEnd2')
run_sample(Solution(), sample3, 'removeNthFromEnd2')
run_sample(Solution(), sample4, 'removeNthFromEnd2')

In [ ]:
# leetcode 020
# https://leetcode.com/problems/valid-parentheses/

sample1='''
"()"
true
'''

sample2='''
"()[]{}"
true
'''

sample3='''
"(]"
false
'''

PAIRS={
    ')': '(',
    ']': '[',
    '}': '{'
}
class Solution:
    # leetcode 020, idea: classic stack-based
    def isValid(self, s: str) -> bool:
        stack=[]
        for c in s:
            if c in PAIRS:
                if len(stack)>0 and stack[-1]==PAIRS[c]:
                    stack.pop(-1) # valid match
                else:
                    return False
            else:
                stack.append(c)
        return len(stack)==0

run_sample(Solution(), sample1, 'isValid')
run_sample(Solution(), sample2, 'isValid')
run_sample(Solution(), sample3, 'isValid')

In [ ]:
# leetcode 021
# https://leetcode.com/problems/merge-two-sorted-lists/

sample1='''
[1,2,4]
[1,3,4]
[1,1,2,3,4,4]
'''

sample2='''
[]
[]
[]
'''

sample3='''
[]
[0]
[0]
'''

# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

class Solution:
    def mergeTwoLists2(self, l1: List[int], l2: List[int]) -> List[int]:
        y=None
        for n in l1[::-1]:
            x=ListNode(n, y)
            y=x
        l1x=y
        y=None
        for n in l2[::-1]:
            x=ListNode(n, y)
            y=x
        l2x=y
        rx=self.mergeTwoLists(l1x, l2x)
        res=[]
        while rx is not None:
            res.append(rx.val)
            rx=rx.next
        return res

    # leetcode 021
    def mergeTwoLists(self, list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:
        res=None
        prev=None
        while not (list1 is None and list2 is None):
            node=list1
            if node is None or (node is not None and list2 is not None and list2.val<node.val):
                node=list2
            if res is None:
                res=node
            if prev is not None:
                prev.next=node
            prev=node
            if node==list1:
                list1=list1.next
            elif node==list2:
                list2=list2.next
        return res

run_sample(Solution(), sample1, 'mergeTwoLists2')
run_sample(Solution(), sample2, 'mergeTwoLists2')
run_sample(Solution(), sample3, 'mergeTwoLists2')

In [ ]:
# leetcode 026
# https://leetcode.com/problems/remove-duplicates-from-sorted-array/

sample1='''
[1,1,2]
[2, [1,2]]
'''

sample2='''
[0,0,1,1,1,2,2,3,3,4]
[5, [0,1,2,3,4]]
'''

class Solution:
    def removeDuplicates2(self, nums: List[int]) -> List:
        k=self.removeDuplicates(nums)
        return [k, nums]

    # leetcode 026
    def removeDuplicates(self, nums: List[int]) -> int:
        j=0
        last=None
        for n in nums:
            if n!=last:
                nums[j]=n
                j+=1
            last=n
        del nums[j:]
        return j

run_sample(Solution(), sample1, 'removeDuplicates2')
run_sample(Solution(), sample2, 'removeDuplicates2')

In [ ]:
# leetcode 036
# https://leetcode.com/problems/valid-sudoku/

sample1='''
[["5","3",".",".","7",".",".",".","."] \
,["6",".",".","1","9","5",".",".","."] \
,[".","9","8",".",".",".",".","6","."] \
,["8",".",".",".","6",".",".",".","3"] \
,["4",".",".","8",".","3",".",".","1"] \
,["7",".",".",".","2",".",".",".","6"] \
,[".","6",".",".",".",".","2","8","."] \
,[".",".",".","4","1","9",".",".","5"] \
,[".",".",".",".","8",".",".","7","9"]]
true
'''

sample2='''
[["8","3",".",".","7",".",".",".","."] \
,["6",".",".","1","9","5",".",".","."] \
,[".","9","8",".",".",".",".","6","."] \
,["8",".",".",".","6",".",".",".","3"] \
,["4",".",".","8",".","3",".",".","1"] \
,["7",".",".",".","2",".",".",".","6"] \
,[".","6",".",".",".",".","2","8","."] \
,[".",".",".","4","1","9",".",".","5"] \
,[".",".",".",".","8",".",".","7","9"]]
false
'''

class Solution:
    # leetcode 036
    def isValidSudoku(self, board: List[List[str]]) -> bool:
        assert len(board)==9
        board=[ [-1 if s=='.' else int(s) for s in row] for row in board ]
        for row in board:
            cs=[n for n in row if n!=-1]
            if not self.valid9set(cs):
                return False
        for col in range(len(board[0])):
            cs=[row[col] for row in board if row[col]!= -1]
            if not self.valid9set(cs):
                return False
        for col in range(0, len(board[0]), 3):
            for rowi in range(0, len(board), 3):
                cs=[]
                for ci2 in range(col, col+3):
                    for ri2 in range(rowi, rowi+3):
                        cell=board[ri2][ci2]
                        if cell!= -1:
                            cs.append(cell)
                if not self.valid9set(cs):
                    return False
        return True
    
    def valid9set(self, nums: List[int]) -> bool:
        if len(nums)>9:
            return False
        for n in nums:
            if n<1 or n>9:
                return False
        if len(set(nums))<len(nums):
            return False
        return True

run_sample(Solution(), sample1, 'isValidSudoku')
run_sample(Solution(), sample2, 'isValidSudoku')

In [ ]:
# leetcode 038
# https://leetcode.com/problems/count-and-say/


sample1='''
1
"1"
'''

sample2='''
4
"1211"
'''

sample3='''
10
"13211311123113112211"
'''

class Solution:
    # leetcode 038, idea: just repeated transforms from left to right through a working string
    def countAndSay(self, n: int) -> str:
        s="1"
        for i in range(n-1):
            news=''
            lastc=None
            cnt=0
            for c in s:
                if c==lastc:
                    cnt+=1
                else:
                    if cnt>0:
                        news+=str(cnt)+lastc
                    lastc=c
                    cnt=1
            if cnt>0:
                news+=str(cnt)+lastc
            s=news
        return s

run_sample(Solution(), sample1, 'countAndSay')
run_sample(Solution(), sample2, 'countAndSay')
run_sample(Solution(), sample3, 'countAndSay')

In [ ]:
# leetcode 048
# https://leetcode.com/problems/rotate-image/

sample1='''
[[1,2,3],[4,5,6],[7,8,9]]
[[7,4,1],[8,5,2],[9,6,3]]
'''

sample2='''
[[5,1,9,11],[2,4,8,10],[13,3,6,7],[15,14,12,16]]
[[15,13,2,5],[14,3,4,1],[12,6,8,9],[16,7,10,11]]
'''

class Solution:
    def rotate2(self, matrix: List[List[int]]) -> List[List[int]]:
        self.rotate(matrix)
        return matrix

    # leetcode 048, idea: start with all items in upper left quadrant,
    # in odd size matrix this should skip the last row,
    # then for each item in upper quadrant copy through in 4 steps
    def rotate(self, matrix: List[List[int]]) -> None:
        sz=len(matrix)
        half=(sz+1)//2
        for row0 in range(half):
            for col0 in range(half):
                if sz%2==1 and row0==half-1:
                    continue # skip last row
                rows=[row0, col0,      sz-1-row0, sz-1-col0, row0]
                cols=[col0, sz-1-row0, sz-1-col0, row0,      col0]
                n=matrix[rows[0]][cols[0]]
                for x in range(1, 5):
                    saved=matrix[rows[x]][cols[x]]
                    matrix[rows[x]][cols[x]]=n
                    n=saved

run_sample(Solution(), sample1, 'rotate2')
run_sample(Solution(), sample2, 'rotate2')

In [ ]:
# leetcode 049
# https://leetcode.com/problems/group-anagrams/

sample1='''
["eat","tea","tan","ate","nat","bat"]
[["bat"], ["eat", "tea", "ate"], ["tan", "nat"]]
'''

sample2='''
[""]
[[""]]
'''

sample3='''
["a"]
[["a"]]
'''

class Solution:
    # leetcode 049, idea: sort letters in each string, then collect duplicates together
    def groupAnagrams(self, strs: List[str]) -> List[List[str]]:
        strs.sort(key=lambda s: ''.join(sorted(s)))
        res=[]
        lastss=None
        for s in strs:
            ss=''.join(sorted(s))
            if lastss is None or ss!=lastss:
                res.append([])
            res[-1].append(s)
            lastss=ss
        return res

run_sample(Solution(), sample1, 'groupAnagrams')
run_sample(Solution(), sample2, 'groupAnagrams')
run_sample(Solution(), sample3, 'groupAnagrams')

In [ ]:
# leetcode 053
# https://leetcode.com/problems/maximum-subarray/

sample1='''
[-2,1,-3,4,-1,2,1,-5,4]
6
'''

sample2='''
[1]
1
'''

sample3='''
[5,4,-1,7,8]
23
'''

sample4='''
[2, 2, -3, 2, -1, -1, 4, -1, 0, 0]
5
'''

sample5='''
[-2,-1,-3]
-1
'''

sample6='''
[-5,8,-5,1,1,-3,5,5,-3,-3,6,4,-7,-4,-8,0,-1,-6]
16
'''

class Solution:
    # leetcode 053, idea: dynamic programming kadane algo,
    # https://en.wikipedia.org/wiki/Maximum_subarray_problem#Kadane's_algorithm
    # basically at any point choose between continuing or restarting the summing at that point,
    # and keeping the best sum
    def maxSubArray(self, nums: List[int]) -> int:
        if len(nums)<1:
            return 0
        best_sum= -(1<<31)
        current_sum = 0
        for n in nums:
            current_sum = max(n, current_sum + n)
            best_sum = max(best_sum, current_sum)
        return best_sum

run_sample(Solution(), sample1, 'maxSubArray')
run_sample(Solution(), sample2, 'maxSubArray')
run_sample(Solution(), sample3, 'maxSubArray')
run_sample(Solution(), sample4, 'maxSubArray')
run_sample(Solution(), sample5, 'maxSubArray')
run_sample(Solution(), sample6, 'maxSubArray')

In [ ]:
# leetcode 066
# https://leetcode.com/problems/plus-one/

sample1='''
[1,2,3]
[1,2,4]
'''

sample2='''
[4,3,2,1]
[4,3,2,2]
'''

sample3='''
[9]
[1,0]
'''

sample4='''
[9,9]
[1,0,0]
'''

class Solution:
    # leetcode 066, idea: start at last digit, as in school
    def plusOne(self, digits: List[int]) -> List[int]:
        i=len(digits)-1
        rest=1
        while True:
            if i<0:
                digits.insert(0, rest)
                break
            digits[i]+=1
            if digits[i]>9:
                digits[i]-=10
                rest=1
                i-=1
            else:
                break
        return digits
        
run_sample(Solution(), sample1, 'plusOne')
run_sample(Solution(), sample2, 'plusOne')
run_sample(Solution(), sample3, 'plusOne')
run_sample(Solution(), sample4, 'plusOne')

In [ ]:
# leetcode 070
# https://leetcode.com/problems/climbing-stairs/

sample1='''
2
2
'''

sample2='''
3
3
'''

sample3='''
1
1
'''

sample4='''
45
1836311903
'''

class Solution:
    # leetcode 070. idea: BFS, with a 'map', recording cumulative ways from a certain previous step
    def climbStairs(self, n: int) -> int:
        mdata={} # tuples of (from-step, to-step) to nr of ways
        todos=[] # heapq of dest. steps that need to be (re)visited
        for i in range(n):
            heapq.heappush(todos, i+1)
        while len(todos)>0:
            dest=heapq.heappop(todos)
            while len(todos)>0 and todos[0]==dest:
                heapq.heappop(todos) # not a set so have to explicitly pop duplicates
            for src in [dest-2, dest-1]:
                if src<0:
                    continue
                elif src==0:
                    ways=1
                else:
                    ways=mdata.get((src-2,src), 0)+mdata.get((src-1,src), 0)
                mdata[(src,dest)]=ways
                if dest+1<=n:
                    heapq.heappush(todos, dest+1)
                if dest+2<=n:
                    heapq.heappush(todos, dest+2)
        #print(n, mdata)
        return mdata.get((n-2,n), 0)+mdata.get((n-1,n), 0)

run_sample(Solution(), sample1, 'climbStairs')
run_sample(Solution(), sample2, 'climbStairs')
run_sample(Solution(), sample3, 'climbStairs')
run_sample(Solution(), sample4, 'climbStairs')

In [ ]:
# leetcode 073
# https://leetcode.com/problems/set-matrix-zeroes/

sample1='''
[[1,1,1],[1,0,1],[1,1,1]]
[[1,0,1],[0,0,0],[1,0,1]]
'''

sample2='''
[[0,1,2,0],[3,4,5,2],[1,3,1,5]]
[[0,0,0,0],[0,4,5,0],[0,3,1,0]]
'''

sample3='''
[[0]]
[[0]]
'''

class Solution:
    def setZeroes2(self, matrix: List[List[int]]) -> List[List[int]]:
        self.setZeroes(matrix)
        return matrix

    # leetcode 073, idea: just maintain sets of rows and cols to be zeroed
    def setZeroes(self, matrix: List[List[int]]) -> None:
        """Do not return anything, modify matrix in-place instead."""
        rows=set()
        cols=set()
        for row_i, row in enumerate(matrix):
            for col_i, v in enumerate(row):
                if v==0:
                    rows.add(row_i)
                    cols.add(col_i)
        for row_i, row in enumerate(matrix):
            for col_i in range(len(row)):
                if row_i in rows or col_i in cols:
                    row[col_i]=0

run_sample(Solution(), sample1, 'setZeroes2')
run_sample(Solution(), sample2, 'setZeroes2')
run_sample(Solution(), sample3, 'setZeroes2')

In [ ]:
# leetcode 088
# https://leetcode.com/problems/merge-sorted-array/

sample1='''
[1,2,3,0,0,0]
3
[2,5,6]
3
[1,2,2,3,5,6]
'''

sample2='''
[1]
1
[]
0
[1]
'''

sample3='''
[0]
0
[1]
1
[1]
'''

class Solution:
    def merge2(self, nums1: List[int], m: int, nums2: List[int], n: int) -> None:
        self.merge(nums1, m, nums2, n)
        return nums1

    # leetcode 088, idea: merge sort from the back
    def merge(self, nums1: List[int], m: int, nums2: List[int], n: int) -> None:
        assert len(nums1)==n+m
        assert len(nums2)==n
        i=len(nums1)-1 # target index in nums1
        j=m-1 # source index in nums1
        k=n-1 # source index in nums2
        while i>=0:
            if j>=0:
                if k>=0 and nums2[k]>nums1[j]:
                    nums1[i]=nums2[k]
                    k-=1
                else:
                    nums1[i]=nums1[j]
                    j-=1
            else:
                assert k>=0
                nums1[i]=nums2[k]
                k-=1
            i-=1

run_sample(Solution(), sample1, 'merge2')
run_sample(Solution(), sample2, 'merge2')
run_sample(Solution(), sample3, 'merge2')

In [ ]:
# leetcode 098
# https://leetcode.com/problems/validate-binary-search-tree/

sample1='''
[2, 1, 3]
true
'''

sample2='''
[5, 1, [4, 3, 6]]
false
'''

sample3='''
[1]
true
'''

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        self.children = []

def readtree(ll: List) -> Optional[TreeNode]:
    root=None
    if ll is not None and isinstance(ll, list):
        if len(ll)<1:
            return root
        root=TreeNode(val=ll[0])
        for cl in ll[1:]:
            root.children.append(readtree(cl))
        if len(root.children)>=1:
            root.left=root.children[0]
        if len(root.children)>=2:
            root.right=root.children[1]
    elif ll is not None: # must be a literal
        root=TreeNode(val=ll)
    return root

class Solution:
    def isValidBST2(self, ll: List) -> bool:
        root=readtree(ll)
        return self.isValidBST(root)

    # leetcode 098
    def isValidBST(self, root: Optional[TreeNode]) -> List:
        res=self.isValidBST1(root)[0]
        return res

    def isValidBST1(self, root: Optional[TreeNode]) -> List:
        '''returns tuple (isValid, minVal, maxVal)'''
        if root is None:
            return (True, None, None)
        if hasattr(root, 'children'):
            assert len(root.children)<=2
        leftRes=self.isValidBST1(root.left)
        rightRes=self.isValidBST1(root.right)
        isValid=leftRes[0] and (leftRes[2] is None or leftRes[2]<root.val) and \
            rightRes[0] and (rightRes[1] is None or rightRes[1]>root.val)
        minVal=root.val
        if leftRes[1] is not None and leftRes[1]<minVal:
            minVal=leftRes[1]
        maxVal=root.val
        if rightRes[2] is not None and rightRes[2]>maxVal:
            maxVal=rightRes[2]
        return (isValid, minVal, maxVal)

run_sample(Solution(), sample1, 'isValidBST2')
run_sample(Solution(), sample2, 'isValidBST2')
run_sample(Solution(), sample3, 'isValidBST2')